In [106]:
import re
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import collections

In [107]:
string = 'abbcccddddEEEEE'
re.sub(r'(\w)\1+', r'\1', string)

'abcdE'

In [108]:
re.sub(r'(\w)\1{1,}', r'\1', string)

'abcdE'

In [109]:
re.sub(r'([a-zA-Z])\1{1,}', r'\1', string)

'abcdE'

In [110]:
''.join(collections.Counter(string).keys())

'abcdE'

In [111]:
inp = ['Mus musculus', 'Agalma elegans', 'Frillagalma vityazi', 'Cordagalma tottoni']
list(map(lambda x: re.sub(r'[a-z]* ', '. ', x), inp))

['M. musculus', 'A. elegans', 'F. vityazi', 'C. tottoni']

In [112]:
inp2 = ['Mus musculus (Y456)', 'Agalma elegans (AB34)', 'Frillagalma vityazi', 'Cordagalma tottoni']
list(map(lambda x: re.sub(r'[a-z]* ', '.', re.sub('\)', '', re.sub(r' \(', '_', x))), inp2))

['M.musculus_Y456', 'A.elegans_AB34', 'F.vityazi', 'C.tottoni']

In [113]:
enzyme = BeautifulSoup(urlopen('https://prosite.expasy.org/PDOC00145').read().decode('utf-8'), 'html.parser')

In [114]:
enz_id = enzyme.find_all('legend', class_='sp_tools')[0].get_text().replace(u'\xa0', u' ')
prosite_id = re.search(r'PS\d*', enz_id).group(0)

In [115]:
fasta_id = 'P0A5H4'
fasta = requests.get(f'https://www.uniprot.org/uniprot/{fasta_id}.fasta').text

In [116]:
fasta

'>sp|P0A5H4|ACEA1_MYCBO Isocitrate lyase 1 OS=Mycobacterium bovis (strain ATCC BAA-935 / AF2122/97) OX=233413 GN=icl PE=3 SV=1\nMSVVGTPKSAEQIQQEWDTNPRWKDVTRTYSAEDVVALQGSVVEEHTLARRGAEVLWEQL\nHDLEWVNALGALTGNMAVQQVRAGLKAIYLSGWQVAGDANLSGHTYPDQSLYPANSVPQV\nVRRINNALQRADQIAKIEGDTSVENWLAPIVADGEAGFGGALNVYELQKALIAAGVAGSH\nWEDQLASEKKCGHLGGKVLIPTQQHIRTLTSARLAADVADVPTVVIARTDAEAATLITSD\nVDERDQPFITGERTREGFYRTKNGIEPCIARAKAYAPFADLIWMETGTPDLEAARQFSEA\nVKAEYPDQMLAYNCSPSFNWKKHLDDATIAKFQKELAAMGFKFQFITLAGFHALNYSMFD\nLAYGYAQNQMSAYVELQEREFAAEERGYTATKHQREVGAGYFDRIATTVDPNSSTTALTG\nSTEEGQFH\n'

In [117]:
seq = re.sub(r'>.*SV=1\n|\n', '', fasta)
seq

'MSVVGTPKSAEQIQQEWDTNPRWKDVTRTYSAEDVVALQGSVVEEHTLARRGAEVLWEQLHDLEWVNALGALTGNMAVQQVRAGLKAIYLSGWQVAGDANLSGHTYPDQSLYPANSVPQVVRRINNALQRADQIAKIEGDTSVENWLAPIVADGEAGFGGALNVYELQKALIAAGVAGSHWEDQLASEKKCGHLGGKVLIPTQQHIRTLTSARLAADVADVPTVVIARTDAEAATLITSDVDERDQPFITGERTREGFYRTKNGIEPCIARAKAYAPFADLIWMETGTPDLEAARQFSEAVKAEYPDQMLAYNCSPSFNWKKHLDDATIAKFQKELAAMGFKFQFITLAGFHALNYSMFDLAYGYAQNQMSAYVELQEREFAAEERGYTATKHQREVGAGYFDRIATTVDPNSSTTALTGSTEEGQFH'

In [118]:
pattern = re.findall(r'Consensus pattern:\n[^n]*\n', enzyme.get_text())[0].strip()
pattern = re.sub('Consensus pattern:\n', '', pattern)
pattern = re.sub('-', '', pattern)
pattern

'K[KR]CGH[LMQR]'

In [119]:
pattern = f'[a-zA-Z]3{pattern}[a-zA-Z]3'
pattern = re.sub('3', '{3}', pattern)

In [120]:
active_site = re.search(pattern, seq)
active_site

<re.Match object; span=(185, 197), match='ASEKKCGHLGGK'>

In [121]:
active_site = re.search(pattern, seq)
pos = active_site.span()
site_seq = active_site.group()
site_seq = site_seq[:3].lower() + site_seq[3:-3] + site_seq[-3:].lower()

In [122]:
f'Isocitrate lyase ({fasta_id}) contains its active site ({prosite_id}) starting from {pos[0] + 3} position: {site_seq}'

'Isocitrate lyase (P0A5H4) contains its active site (PS00161) starting from 188 position: aseKKCGHLggk'